In [1]:
#Preliminary predictive model for clothing predictions

In [2]:
import numpy as np
import pandas as pd
import random
import datetime

In [3]:
#Creating the dataset

#features

"""
Weather features:
1. Temperature (Celcius) (-20 to 25)
2. Humidity (%) (relative) (0 to 100)
3. Probability of Precipitation (%) (0 to 100)
4. Wind speed (mph)

User features:
1. Age (7 to 70)
2. Weight (kg)
3. Sex
4. Height (cm)
5. Fat percentage
"""

#output
"""
output will be in the form of a boolean array with each index presenting different clothing articles
0. short sleve shirt + long pants (always on)
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""

'\noutput will be in the form of a boolean array with each index presenting different clothing articles\n0. short sleve shirt + long pants (always on)\n1. Thermal (both top and bottom)\n2. Jacket/Hoodie\n3. light downjacket\n4. Thick down/Winter down\n5. Wind breaker\n6. Umbrella/Raincoat\n'

In [4]:
from datetime import date
data = []
currentdate = date.today()
for i in range(100000):
    date = (currentdate - datetime.timedelta(i)).strftime("%d/%m/%Y") #display in day/month/year
    temperature = random.randint(0, 45) - 20
    humidity = random.randint(0, 100)
    precipitation = random.randint(0, 100)
    windspeed = random.randint(0, 40) #mph
    age = random.randint(7, 70)
    weight = random.randint(20, 120) #kg
    height = random.randint(110, 200) #cm
    sex = random.randint(0,1) # 0:M 1:F
    #fat percentage formula from
    #https://www.gaiam.com/blogs/discover/how-to-calculate-your-ideal-body-fat-percentage
    
    BMI = weight/(height**2)*10000
    if sex == 0:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 16.2
        
    elif sex == 1:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 5.4
    
    #adjustment for negative fatpercentage
    if fatpercentage < 0:
        fatpercentage = 1
        
    datapoint = (date, temperature, humidity, precipitation, windspeed, age, weight, height, sex, fatpercentage, BMI)
    data.append(datapoint)
    

In [5]:
columnstring = 'date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi'
column = columnstring.split(',')
df = pd.DataFrame(data, columns = column)

In [6]:
#df[:100]

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi
0,11/01/2022,-11,80,37,32,52,35,157,0,12.799231,14.199359
1,10/01/2022,-2,35,61,2,58,74,191,1,32.281438,20.284532
2,09/01/2022,25,81,20,17,48,72,170,1,35.536194,24.913495
3,08/01/2022,13,55,17,23,32,35,158,0,7.984227,14.020189
4,07/01/2022,7,34,67,35,13,42,162,1,16.794390,16.003658
...,...,...,...,...,...,...,...,...,...,...,...
95,08/10/2021,-15,21,6,5,55,54,171,0,18.610665,18.467221
96,07/10/2021,-9,77,10,6,42,27,170,1,15.471073,9.342561
97,06/10/2021,20,58,91,31,61,117,197,0,34.007175,30.147646
98,05/10/2021,16,53,34,3,19,86,114,1,78.379049,66.174207


In [7]:
#layers

"""
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""




#cold resistance calculation

male_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}


female_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}

# cold resistance categories:
# low: 0 - 24
# mid-low: 25 - 49
# mid-high: 50 - 74
# high: 75 - 99
cold_resistance_scores = {
    'low' : 25,
    'mid-low' : 50,
    'mid-high' : 75,
    'high' : 100
}




In [8]:
#returns cold resistance score
#formula:

#cold_resistance = age_score * gender_age_weight + bmi_score + gender_bmi_weight + gender_fatpercentage_weight
def coldresistance(sex, age, fatpercentage, BMI):
    #female case
    if sex == 1:
        if (age <= 12 or age >= 55):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 13 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 14):
            fatpercentage_score = cold_resistance_scores['low']
        elif (14 < fatpercentage <= 20):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (20 < fatpercentage <= 25): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
      
    #men
    else:
        if (age <= 12 or age >= 55):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 13 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 8):
            fatpercentage_score = cold_resistance_scores['low']
        elif (8 < fatpercentage <= 15):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (15 < fatpercentage <= 24): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
            
    #female case
    if sex == 1:
        cold_resistance = age_score/100*female_weights['age'] + bmi_score/100*female_weights['bmi'] + fatpercentage_score/100*female_weights['fatpercentage']
    
    #male case
    elif sex == 0:
        cold_resistance = age_score/100*male_weights['age'] + bmi_score/100*male_weights['bmi'] + fatpercentage_score/100*male_weights['fatpercentage']
    
    return cold_resistance

In [9]:
df['cold_resistance'] = df.apply(lambda row : coldresistance(row['sex'], row['age'], row['fatpercentage'], row['bmi']), axis = 1)

In [10]:
#df[df['cold_resistance'] > 99]

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance
8,03/01/2022,20,55,20,5,26,112,118,0,86.303987,80.436656,100.0
12,30/12/2021,-9,86,87,25,30,118,141,0,61.923781,59.353151,100.0
15,27/12/2021,22,65,83,22,28,109,155,0,44.683288,45.369407,100.0
38,04/12/2021,5,94,34,0,32,114,148,0,53.614346,52.045289,100.0
49,23/11/2021,-16,2,58,34,27,102,162,1,47.449232,38.866027,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
99969,28/04/1748,15,32,21,23,32,92,133,0,53.571668,52.009724,100.0
99970,27/04/1748,-20,23,67,6,33,66,125,0,42.078000,42.240000,100.0
99971,26/04/1748,10,87,50,27,30,35,111,1,35.588142,28.406785,100.0
99984,13/04/1748,-8,39,46,22,29,96,137,1,62.647804,51.148170,100.0


In [13]:
#rules
"""
1. If temp < 5 : Thermal = True and thickdown = True
2. If temp < 15 : Jacket = True
3. If temp < 10 : lightdown = True
4. If windspeed >= 25 : windbreaker = True
5. If precipitation > 50%: umbrella/raincoast = True


cold resistance categories:
low: 0 - 24
mid-low: 25 - 49
mid-high: 50 - 74
high: 75 - 99
"""

clothesmap = {
    'thermal' : 0, #(both top and bottom)
    'hoodie' : 1,
    'light_down' : 2,
    'thick_down' : 3,
    'wind_breaker' : 4,
    'umbrella' : 5
}

def clothingPredictor(coldresistance, temperature, humidity, precipitation, windspeed):
    #prediction is an array of booleans with each index corresponding to a certain clothing article
    prediction = [False] * 6
    
    
    #wind breaker and umbrella are independent of cold resistance
    if windspeed >= 25:
        prediction[clothesmap['wind_breaker']] = True
    
    if precipitation >= 50:
        prediction[clothesmap['umbrella']] = True
    
    
    #low cold resistance
    if coldresistance <= 25:
        #print('low cr')
        if 20 < temperature <= 25:
            prediction[clothesmap['hoodie']] = True
            
        
        elif 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            
        elif temperature < 10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            
        #if temp > 25 dont put on any additional layers
        else:
            pass
            
    #mid-low cold resistance
    elif 25 < coldresistance <= 50:
        #print('mid- low cr')
        if 20 < temperature <= 25:
            prediction[clothesmap['hoodie']] = True
            
        elif 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif temperature <= 5:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
        #if temp > 25 dont put on any additional layers
        else:
            pass
        
    #mid-high cold resistance
    elif 50 < coldresistance <= 75:
        #print('mid - high cr')
        if 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 0 < temperature <= 5:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif temperature <= 0:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
        #if temp > 20 dont put on any additional layers
        else:
            pass
        
    #high cold resistance
    elif coldresistance > 75:
        #print('high cr')
        if 10 <= temperature <= 15:
            prediction[clothesmap['hoodie']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['hoodie']] = True
            
        elif 0 < temperature <= 5:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif -10 < temperature <= 0:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif temperature <= -10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            #print('yay')
        #if temp > 15 dont put on any additional layers
        else:
            pass
        
    
    
    return prediction

In [14]:
for article in clothesmap:
    df[article] = df.apply(lambda row : clothingPredictor(row['cold_resistance'], row['temperature'], row['humidity'],\
                                                          row['precipitation'], row['windspeed'])[clothesmap[article]], axis = 1)

In [15]:
#df.head()

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance,thermal,hoodie,light_down,thick_down,wind_breaker,umbrella
0,11/01/2022,-11,80,37,32,52,35,157,0,12.799231,14.199359,52.5,True,True,True,True,True,False
1,10/01/2022,-2,35,61,2,58,74,191,1,32.281438,20.284532,62.5,True,True,True,True,False,True
2,09/01/2022,25,81,20,17,48,72,170,1,35.536194,24.913495,82.5,False,False,False,False,False,False
3,08/01/2022,13,55,17,23,32,35,158,0,7.984227,14.020189,55.0,False,True,True,False,False,False
4,07/01/2022,7,34,67,35,13,42,162,1,16.794390,16.003658,50.0,True,True,True,False,True,True


In [16]:
#print(clothingPredictor(80, -18, 0, 12, 12))

[True, True, True, True, False, False]
